In [1]:
import torch
import torch.nn as nn

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, BertConfig

from datasets import load_dataset
import numpy as np

from captum.attr import IntegratedGradients, GradientShap
from captum.attr import InterpretableEmbeddingBase, TokenReferenceBase
from captum.attr import visualization
from captum.attr import configure_interpretable_embedding_layer, remove_interpretable_embedding_layer

In [2]:
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")

In [3]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained("../models/bert_imdb.pt")

In [4]:
def compute_bert_outputs(model_bert, embedding_output, attention_mask=None, head_mask=None):
    if attention_mask is None:
        attention_mask = torch.ones(embedding_output.shape[0], embedding_output.shape[1]).to(embedding_output)

    extended_attention_mask = attention_mask.unsqueeze(1).unsqueeze(2)

    extended_attention_mask = extended_attention_mask.to(dtype=next(model_bert.parameters()).dtype) # fp16 compatibility
    extended_attention_mask = (1.0 - extended_attention_mask) * -10000.0

    if head_mask is not None:
        if head_mask.dim() == 1:
            head_mask = head_mask.unsqueeze(0).unsqueeze(0).unsqueeze(-1).unsqueeze(-1)
            head_mask = head_mask.expand(model_bert.config.num_hidden_layers, -1, -1, -1, -1)
        elif head_mask.dim() == 2:
            head_mask = head_mask.unsqueeze(1).unsqueeze(-1).unsqueeze(-1)  # We can specify head_mask for each layer
        head_mask = head_mask.to(dtype=next(model_bert.parameters()).dtype) # switch to fload if need + fp16 compatibility
    else:
        head_mask = [None] * model_bert.config.num_hidden_layers

    encoder_outputs = model_bert.encoder(embedding_output,
                                         extended_attention_mask,
                                         head_mask=head_mask)
    sequence_output = encoder_outputs[0]
    pooled_output = model_bert.pooler(sequence_output)
    outputs = (sequence_output, pooled_output,) + encoder_outputs[1:]  # add hidden_states and attentions if they are here
    return outputs  # sequence_output, pooled_output, (hidden_states), (attentions)    


In [5]:
class BertModelWrapper(nn.Module):
    
    def __init__(self, model):
        super(BertModelWrapper, self).__init__()
        self.model = model
        
    def forward(self, embeddings):        
        outputs = compute_bert_outputs(self.model.bert, embeddings)
        pooled_output = outputs[1]
        pooled_output = self.model.dropout(pooled_output)
        logits = self.model.classifier(pooled_output)
        return torch.softmax(logits, dim=1)[:, 1].unsqueeze(1)

In [6]:
bert_model_wrapper = BertModelWrapper(model)
ig = IntegratedGradients(bert_model_wrapper)

# accumalate couple samples in this array for visualization purposes
vis_data_records_ig = []

In [25]:
def interpret_sentence(model_wrapper, sentence, label=1, ig_model=True):
    #model.to(device)
    model_wrapper.eval()
    model_wrapper.zero_grad()
    
    input_ids = torch.tensor([tokenizer.encode(sentence, add_special_tokens=True)], device=device)
    input_embedding = model_wrapper.model.bert.embeddings(input_ids)
    
    ref_token_id = tokenizer.pad_token_id # A token used for generating token reference
    sep_token_id = tokenizer.sep_token_id # A token used as a separator between question and text and it is also added to the end of the text.
    cls_token_id = tokenizer.cls_token_id # A token used for prepending to the concatenated question-text word sequence

    ref_input_ids = [cls_token_id] + (input_ids.size(1)-2) * [ref_token_id] + [sep_token_id]
    ref_input_ids = torch.tensor([ref_input_ids], device=device)
    
    print(input_ids.size())
    print(ref_input_ids.size())
    ref_embedding = model_wrapper.model.bert.embeddings(ref_input_ids)

    # predict
    pred = model_wrapper(input_embedding).item()
    pred_ind = round(pred)
    
    # compute attributions and approximation delta using integrated gradients
    if ig_model:
        attributions_exp, delta = ig.attribute(input_embedding, n_steps=500, 
                                              baselines=ref_embedding,
                                              return_convergence_delta=True)
        print('pred: ', pred_ind, '(', '%.2f' % pred, ')', ', delta: ', abs(delta))
        print(attributions_exp)
        tokens = tokenizer.convert_ids_to_tokens(input_ids[0].cpu().clone().numpy().tolist())    
        add_attributions_to_visualizer(attributions_exp, tokens, pred, pred_ind, label, delta, vis_data_records_ig)
    else:
        attributions_exp, delta = ig.attribute(input_embedding, n_samples=500,
                                              baselines=ref_embedding,
                                              return_convergence_delta=True)
        print('pred: ', pred_ind, '(', '%.2f' % pred, ')')
       
        tokens = tokenizer.convert_ids_to_tokens(input_ids[0].cpu().clone().numpy().tolist())    
        add_attributions_to_visualizer(attributions_exp, tokens, pred, pred_ind, label, delta, vis_data_records_ig)


In [29]:
def add_attributions_to_visualizer(attributions, tokens, pred, pred_ind, label, delta, vis_data_records):
    attributions = attributions.sum(dim=2).squeeze(0)
    #attributions = attributions / torch.norm(attributions)
    attributions = attributions.detach().cpu().clone().numpy()
    
    # storing couple samples in an array for visualization purposes
    print(attributions)
    vis_data_records.append(visualization.VisualizationDataRecord(
                            attributions,
                            pred,
                            pred_ind,
                            label,
                            "label",
                            attributions.sum(),       
                            tokens[:len(attributions)],
                            delta))    

In [9]:
def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True)

_, test_dataset = load_dataset('imdb', split=['train', 'test'])
test_dataset = test_dataset.map(tokenize, batched=True, batch_size=len(test_dataset))
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

Reusing dataset imdb (/.cache/huggingface/datasets/imdb/plain_text/1.0.0/90099cb476936b753383ba2ae6ab2eae419b2e87f71cd5189cb9c8e5814d12a3)
Loading cached processed dataset at /.cache/huggingface/datasets/imdb/plain_text/1.0.0/90099cb476936b753383ba2ae6ab2eae419b2e87f71cd5189cb9c8e5814d12a3/cache-2d50dc0059a76e34.arrow


In [13]:
r = np.random.choice(len(test_dataset))
label = test_dataset['label'][r].item()
sentence = test_dataset['text'][r]

In [14]:
new = "In tokenization, text is splitted."
tokenizer.tokenize(new)

['in', 'token', '##ization', ',', 'text', 'is', 'split', '##ted', '.']

In [15]:
interpret_sentence(bert_model_wrapper, sentence=sentence, label=label)
visualization.visualize_text(vis_data_records_ig)

torch.Size([1, 377])
torch.Size([1, 377])
pred:  1 ( 0.99 ) , delta:  tensor([0.0019], dtype=torch.float64)


In [34]:
ig = GradientShap(bert_model_wrapper)

# accumalate couple samples in this array for visualization purposes
vis_data_records_ig = []

interpret_sentence(bert_model_wrapper, sentence='it was the best film ever', label=1, ig_model=False)
visualization.visualize_text(vis_data_records_ig)

torch.Size([1, 8])
torch.Size([1, 8])
pred:  1 ( 1.00 )
[ 0.          0.12418355 -0.05660779  0.1500635   0.25705305  0.047584
  0.07372487  0.        ]


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,1 (1.00),label,0.60,[CLS] it was the best film ever [SEP]


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,1 (1.00),label,0.60,[CLS] it was the best film ever [SEP]


In [13]:
ig = IntegratedGradients(bert_model_wrapper)

# accumalate couple samples in this array for visualization purposes
vis_data_records_ig = []
r=100
label = test_dataset['label'][r].item()
sentence = test_dataset['text'][r]
interpret_sentence(bert_model_wrapper, sentence=sentence, label=label)
visualization.visualize_text(vis_data_records_ig)

torch.Size([1, 381])
torch.Size([1, 381])
pred:  1 ( 0.99 ) , delta:  tensor([0.0027], dtype=torch.float64)


In [15]:
ig = GradientShap(bert_model_wrapper)

# accumalate couple samples in this array for visualization purposes
vis_data_records_ig = []

interpret_sentence(bert_model_wrapper, sentence=sentence, label=label, ig_model=False)
visualization.visualize_text(vis_data_records_ig)

torch.Size([1, 381])
torch.Size([1, 381])
pred:  1 ( 0.99 )
